In [77]:
import os
import cv2
import json
import tqdm
import numpy as np
from multiprocessing import Pool
from IPython.display import display
from matplotlib import pyplot as plt

In [78]:
# use os.walk to get all pngs in a directory
def calc_nonzero_porportion(image_path, use_jpg=False, show=False):
    if use_jpg: image_path = image_path.replace('png', 'jpg')
    parsing_result = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if show:
        plt.imshow(parsing_result)
        plt.show()
    non_zero_count = np.count_nonzero(parsing_result)
    porportion = non_zero_count / parsing_result.size
    return porportion   


In [79]:
root_dir = '/data2/chence/multi_view_hq'
assert os.path.exists(os.path.join(root_dir, 'dataset.json'))
with open(os.path.join(root_dir, 'dataset.json'), 'r') as f:
    dataset = json.load(f)
    print(f'load dataset.json from {root_dir} with {len(dataset)} images')

load dataset.json with 268807 images


In [80]:
thresh = 0.3
backet = 0.001
vis_sample = 20
output_dict = {}
output_filtered_dict = {}
output_json_path = os.path.join(root_dir, f'dataset_thresh.json')
output_filtered_json_path = os.path.join(root_dir, f'dataset_thresh_{thresh}.json')

In [81]:
for image_name, image_meta in tqdm.tqdm(dataset.items()):
    image_path = os.path.join(root_dir, 'align_parsing',image_name)
    ratio = calc_nonzero_porportion(image_path)
    output_dict[image_name] = ratio
    if ratio > thresh:
        output_filtered_dict[image_name] = ratio

100%|██████████| 268807/268807 [08:19<00:00, 537.63it/s]


In [84]:
if not(os.path.exists(output_json_path)):
    with open(output_json_path, 'w') as f:
        json.dump(output_dict, f, indent=4)

with open(output_filtered_json_path, 'w') as f:
    json.dump(output_filtered_dict, f, indent=4)

print(f'Saved to {output_json_path} and {output_filtered_json_path}')
filtered_values = list(output_filtered_dict.values())
assert min(filtered_values) > thresh

Saved to /data2/chence/multi_view_hq/dataset_thresh.json and /data2/chence/multi_view_hq/dataset_thresh_0.3.json


In [83]:

    # if  p < thresh + backet and p > thresh - backet:
    #     print(im_path, p)
    #     orgn_image_path = images[idx].replace('align_parsing', 'align_images')
    #     orgn_image = Image.open(orgn_image_path)
    #     parsing_result = Image.open(images[idx])
    #     # resize, make height = 128
    #     h, w = parsing_result.size
    #     parsing_result = parsing_result.resize((int(128 * w / h), 128))
    #     orgn_image = orgn_image.resize((int(128 * w / h), 128))
    #     fig, ax = plt.subplots(1, 2, figsize=(4, 4))
    #     ax[0].imshow(orgn_image)
    #     ax[1].imshow(parsing_result)
    #     plt.show()
    #     vis_sample -= 1
    #     if vis_sample == 0: break
        # display(Image.open(images[idx].replace('align_parsing', 'align_images')))
        # display(Image.open(images[idx]))